In [15]:
#User Reviews
#Importo Librerias
#!pip install textblob
import pandas as pd
import ast
import gzip
from textblob import TextBlob


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ------------------------------------- 636.8/636.8 kB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 7.4 MB/s eta 0:00:00
     ------------------------------------ 269.5/269.5 kB 753.6 kB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 97.9/97.9 kB 2.8 MB/s eta 0:00:00


In [16]:
# Asigno al DataFrame el archivo json user_reviews.json.gz
Lista_Reviews = []
#with gzip.open(u"/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/user_reviews.json.gz", 'rt', encoding='UTF-8') as f:
with gzip.open(u"./Dataset/user_reviews.json.gz", 'rt', encoding='UTF-8') as f:
    for line in f.readlines():
        #Usamos ast.literal_eval para convertir cada línea en un diccionario y lo agregamos a la lista filas.
        Lista_Reviews.append(ast.literal_eval(line))

df_ur = pd.DataFrame(Lista_Reviews)
df_ur.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [12]:
# Duplico las filas generando un diccionario por cada dicc#ionario en la lista
df_ur = df_ur.explode('reviews') 
# Ahora concateno el dataframe original, con el dataframe generado a partir de transformar los diccionarios a pandas
df_ur = pd.concat([df_ur.drop(['reviews'],axis=1),df_ur['reviews'].apply(pd.Series)],axis=1).reset_index()
df_ur.head()

,index,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
3,1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
4,1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN


In [13]:
#Elimino columnas
df_ur.drop(columns=['user_url','funny','helpful','last_edited',0,'index'],inplace=True)
df_ur.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


In [91]:
#
user_review_explode

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,True,:D


In [92]:
#from textblob import TextBlob

# Assuming user_review_explode is your DataFrame, and 'reviews' is the column containing the text reviews
df_ur['sentiment_analysis'] = df_ur['review'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)

# Map sentiment scores to the specified scale (0 for bad, 1 for neutral, and 2 for positive)
df_ur['sentiment_analysis'] = df_ur['sentiment_analysis'].map(lambda x: 0 if x < 0 else (1 if x == 0 else 2))

In [93]:
#
user_review_explode

,user_id,posted,item_id,recommend,review,sentiment_analysis
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,2
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,2
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,0
...,...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...,2
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...,2
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,2
59331,LydiaMorley,Posted July 20.,730,True,:D,2


In [94]:
#
user_review_explode.isnull().sum()

user_id                0
posted                28
item_id               28
recommend             28
review                28
sentiment_analysis     0
dtype: int64

In [95]:
#
user_review_explode[user_review_explode.isnull().any(axis=1)]

,user_id,posted,item_id,recommend,review,sentiment_analysis
137,gdxsd,NaN,NaN,NaN,NaN,1
177,76561198094224872,NaN,NaN,NaN,NaN,1
2559,76561198021575394,NaN,NaN,NaN,NaN,1
10080,cmuir37,NaN,NaN,NaN,NaN,1
13767,Jaysteeny,NaN,NaN,NaN,NaN,1
15493,ML8989,NaN,NaN,NaN,NaN,1
19184,76561198079215291,NaN,NaN,NaN,NaN,1
20223,76561198079342142,NaN,NaN,NaN,NaN,1
25056,76561198061996985,NaN,NaN,NaN,NaN,1
26257,76561198108286351,NaN,NaN,NaN,NaN,1


In [96]:
#
user_review_explode.dropna(inplace=True)

In [97]:
user_review_explode


,user_id,posted,item_id,recommend,review,sentiment_analysis
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,2
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,2
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,0
...,...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...,2
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...,2
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,2
59331,LydiaMorley,Posted July 20.,730,True,:D,2


In [84]:
user_review_explode[user_review_explode['sentiment_analysis']==2]

,index,user_id,posted,item_id,recommend,review,sentiment_analysis
0,0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,2
1,0,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,2
2,0,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,2
3,1,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,2
6,2,evcentric,Posted February 3.,248820,True,A suitably punishing roguelike platformer. Wi...,2
...,...,...,...,...,...,...,...
59328,25797,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...,2
59329,25797,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...,2
59330,25798,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,2
59331,25798,LydiaMorley,Posted July 20.,730,True,:D,2


In [99]:
#
#user_review_explode.to_csv('../datasets/df_ur.csv',index=False)
user_review_explode.to_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_ur.csv',index=False)

In [ ]:
# def PlayTimeGenre( genero : str ):
df_sg

,item_id,genres,app_name,release_date
0,761140,Action,Lost Summoner Kitty,2018
1,761140,Casual,Lost Summoner Kitty,2018
2,761140,Indie,Lost Summoner Kitty,2018
3,761140,Simulation,Lost Summoner Kitty,2018
4,761140,Strategy,Lost Summoner Kitty,2018
...,...,...,...,...
70694,610660,Indie,Russian Roads,2018
70695,610660,Racing,Russian Roads,2018
70696,610660,Simulation,Russian Roads,2018
70697,658870,Casual,EXIT 2 - Directions,2017


In [ ]:
#
#user_review_explode = df_ur.explode('reviews') # Duplico las filas generando un diccionario por cada dicc#ionario en la lista
#user_review_explode
#df_ur['reviews'][0][76561197970982479	]
# Ahora concateno el dataframe original, con el dataframe generado a partir de transformar los diccionarios a pandas
#user_review_explode = pd.concat([user_review_explode.drop(['reviews'],axis=1),user_review_explode['reviews'].apply(pd.Series)],axis=1)

df_ur['user_id'][0][0][0]


0    76561197970982479
0    76561197970982479
0    76561197970982479
Name: user_id, dtype: object

In [ ]:
 # hago un merge teniendo todos los usuarios
user_rev_item = pd.merge(df_ui[['user_id','item_id','playtime_forever']],user_review[['user_id','recommend']],on = 'user_id',how='left')
# Me quedo solo con los que estan nulos en reviews
user_rev_item = user_rev_item[user_rev_item['recommend'].isnull()]
# Hago un agrupamiento por usuario y saco el id del que tiene mas minuntos.
max_playtime_items = user_rev_item.groupby('user_id')['playtime_forever'].idxmax()
result_df = user_rev_item.loc[max_playtime_items, ['user_id', 'item_id']]
result_df.to_csv('../datasets/user_item_max_horas.csv',index=False) # Exporto para ser usado por la funcion

In [ ]:
def verify_date_format(df, column_name):
    try:
        pd.to_datetime(df[column_name], format='%Y-%m-%d', errors='raise')
        return True  # The format is 'YYYY-MM-DD'
    except ValueError:
        return False  # The format is not 'YYYY-MM-DD'


In [ ]:
if verify_date_format(df_sg, 'release_date'):
    print(f"The format of '{'release_date'}' is 'YYYY-MM-DD'")
else:
    print(f"The format of '{'release_date'}' is not 'YYYY-MM-DD'")

In [ ]:
import pandas as pd
from dateutil import parser

def filter_dataframe_by_year_format(df, column_name):
    # Function to extract the year from various date formats
    def extract_year(date_str):
        try:
            parsed_date = parser.parse(str(date_str))
            return parsed_date.year
        except ValueError:
            return None

    # Apply the extract_year function to the specified column
    df['extracted_year'] = df['release_date'].apply(extract_year)

    # Filter rows with the correct 'YYYY-MM-DD' format or extracted year
    mask = (df['release_date'].astype(str).str.match(r'^\d{4}-\d{2}-\d{2}$')) | (~df['extracted_year'].isnull())

    # If you want to filter based on a specific year, you can modify the following line
    # For example, to filter for the year 2022, use: mask = mask & ((df['extracted_year'] == 2022) | (df[column_name].astype(str).str.match(r'^\d{4}-\d{2}-\d{2}$')))

    # Drop the temporary 'extracted_year' column
    df = df.drop(columns=['extracted_year'])

    filtered_df = df[mask]

    return filtered_df

# Example usage:
# Assuming df is your pandas DataFrame and 'date_column' is the column you want to filter
# Replace 'date_column' with the actual column name in your DataFrame
filtered_dataframe = filter_dataframe_by_year_format(df, 'date_column')

In [ ]:
import pandas as pd
from dateutil import parser

def extract_years_from_column(df, column_name):
    # Function to extract the year from various date formats
    def extract_year(date_str):
        try:
            parsed_date = parser.parse(str(date_str))
            return parsed_date.year
        except ValueError:
            return None

    # Apply the extract_year function to the specified column
    df['extracted_year'] = df['release_date'].apply(extract_year)

    return df['extracted_year']

# Example usage:
# Assuming df is your pandas DataFrame and 'date_column' is the column you want to extract years from
# Replace 'date_column' with the actual column name in your DataFrame

# Extract years and create a mask for valid years
years_mask = extract_years_from_column(df_sg, 'release_date').notnull()

# Apply the mask to filter the DataFrame
filtered_dataframe = df_sg[years_mask]

In [ ]:
filtered_dataframe.info()
filtered_dataframe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29893 entries, 88310 to 120443
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       23897 non-null  object 
 1   genres          28659 non-null  object 
 2   app_name        29892 non-null  object 
 3   title           29892 non-null  object 
 4   url             29893 non-null  object 
 5   release_date    29893 non-null  object 
 6   tags            29732 non-null  object 
 7   reviews_url     29893 non-null  object 
 8   specs           29224 non-null  object 
 9   price           28816 non-null  object 
 10  early_access    29893 non-null  float64
 11  id              29893 non-null  float64
 12  developer       28643 non-null  object 
 13  extracted_year  29893 non-null  float64
dtypes: float64(3), object(11)
memory usage: 3.4+ MB


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,extracted_year
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,2018.0
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL,2018.0
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com,2017.0
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域,2017.0
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0.0,772540.0,Trickjump Games Ltd,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,0.0,745400.0,Bidoniera Games,2018.0
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS""",2018.0
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada,2018.0
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich,2018.0


In [ ]:
import ast
import gzip
users_items_gz = "/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/users_items.json.gz"
filas=[]
with gzip.open(users_items_gz, 'rt', encoding='MacRoman') as archivo:
    for line in archivo.readlines():
        filas.append(ast.literal_eval(line))

df_ui = pd.DataFrame(filas)
df_ui = df_ui.explode('items').reset_index()
df_ui = df_ui.drop(columns="index")
df_ui = pd.concat([df_ui, pd.json_normalize(df_ui['items'])], axis=1)
df_ui = df_ui.drop(columns='items')


In [ ]:
df_ui

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,519140,Minds Eyes,3.0,3.0


In [ ]:
#steam_proc['tags'] = steam_proc['tags'].apply(lambda x: [item for item in x if item in genres] if isinstance(x, list) else x)
#steam_proc.loc[valid_date_mask, 'Year'] = pd.to_datetime(steam_proc.loc[valid_date_mask, 'release_date'], errors='coerce').dt.year
#steam_proc['Year'] = steam_proc['Year'].astype(float)

In [ ]:
import gzip
import json
import ast
import shutil

# Nueva sección

In [ ]:
#df_ur=pd.read_json('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/user_reviews.json.gz',orient='JsonSeriesOrient',lines = True,encoding='utf-8-sig')
#df_ur=pd.read_json('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/user_reviews.json.gz',orient='split',lines = True,encoding='utf-8-sig')
user_reviews_gz = "/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/user_reviews.json.gz"
filas=[]
with gzip.open(user_reviews_gz, 'rt', encoding='MacRoman') as archivo:
    for line in archivo.readlines():
        filas.append(ast.literal_eval(line))

user_review = pd.DataFrame(filas)

In [ ]:
user_review

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [ ]:
#import ast as ast
#Asigno al DataFrame el archivo json user_reviews.json.gz
#df_ur=pd.read_json('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/user_reviews.json.gz',orient='JsonSeriesOrient',lines = True,encoding='utf-8-sig')
#df_ur=pd.read_json('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/user_reviews.json.gz',lines = False,encoding='utf-8-sig')

#archivo = '/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/user_reviews.json.gz'
#lista para guardar cada fila
#lista = []
#with open(archivo, encoding='utf-8') as file:
#    for line in file.readlines():
#        lista.append(ast.literal_eval(line))

#def abrir_json_comprimido(ruta_archivo):
#	with gzip.open(ruta_archivo,'rt',encoding='utf-8') as archivo_comprimido:
#		contenido_json=json.load(archivo_comprimido)
#	return contenido_json
#Ejemplo de uso
#ruta_archivo = '/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/user_reviews.json.gz'
#contenido= abrir_json_comprimido(ruta_archivo)

#Ahora puedes trabajar con el contenido como un diccionario de Python
#print(contenido)

# Descomprimo los archivos

# Ruta al archivo comprimido
ruta_comprimida_ur= '/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/user_reviews.json.gz'

# Ruta para guardar temporalmente el archivo descomprimido en Colab
ruta_temporal_ur = '/content/user_reviews.json'

# Descomprimir el archivo
with gzip.open(ruta_comprimida_ur, 'rb') as f_in:
    with open(ruta_temporal_ur, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
# Guardo en una variable el archivo
data_ur = '/content/user_reviews.json'

# Cargo archivo JSON línea por línea y almaceno en una lista.

# Lista para guardar cada fila
lista_ur = []

with open(data_ur, encoding='utf-8') as file:
    for line in file.readlines():
            lista_ur.append(ast.literal_eval(line))

# Guardo el archivo en un DataFrame
df_ur = pd.DataFrame(lista_ur)
df_ur.head(5)


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [ ]:
#Visualizo el DataFrame
df_ur

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [ ]:
users_items_gz = "/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/users_items.json.gz"
filas=[]
with gzip.open(users_items_gz, 'rt', encoding='MacRoman') as archivo:
    for line in archivo.readlines():
        filas.append(ast.literal_eval(line))

users_items = pd.DataFrame(filas)

In [ ]:
users_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [ ]:
users_items['items'][0][1]

{'item_id': '20',
 'item_name': 'Team Fortress Classic',
 'playtime_forever': 0,
 'playtime_2weeks': 0}

In [ ]:
max_playtime_items = user_rev_item.groupby('user_id')['playtime_forever'].idxmax()

In [ ]:
#Asigno al DataFrame el archivo json users_items.json.gz
#df_ui=pd.read_json('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/users_items.json.gz',lines=True)

# Ruta al archivo comprimido
ruta_comprimida_ui= '/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/users_items.json.gz'

# Ruta para guardar temporalmente el archivo descomprimido en Colab
ruta_temporal_ui = '/content/users_items.json'

# Descomprimir el archivo
with gzip.open(ruta_comprimida_ui, 'rb') as f_in:
    with open(ruta_temporal_ui, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
# Guardo en una variable el archivo
data_ui = '/content/users_items.json'

# Cargo archivo JSON línea por línea y almaceno en una lista.

# Lista para guardar cada fila
lista_ui = []

with open(data_ui, encoding='utf-8') as file:
    for line in file.readlines():
            lista_ui.append(ast.literal_eval(line))

# Guardo el archivo en un DataFrame
df_ui = pd.DataFrame(lista_ur)
df_ui.head(5)


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
